In [ ]:
device = 'cuda:3'

In [11]:
from data_loader import get_mix_instruct

prompts, references, ds_name = get_mix_instruct("train", 21000)

Dataset: mix-instruct_train_21000 found in cache, loading from cache ✅


In [12]:
from utility_functions.delift_se import get_delift_se_utility

utility, utility_name = get_delift_se_utility(prompts, references, ds_name)

Utility: mix-instruct_train_21000_delift-se found in cache, loading from cache ✅


In [13]:
from subset import create_subset, get_subset

subset, subset_name = create_subset(utility, utility_name)

Subset: mix-instruct_train_21000_delift-se_0.3 found in cache, loading from cache ✅


In [ ]:
from inference import generate_responses

# Phi3: 'microsoft/Phi-3-mini-128k-instruct'
# LLama: 'meta-llama/Llama-3.2-3B'
responses, generation_name = generate_responses(prompts, 'microsoft/Phi-3-mini-128k-instruct', ds_name, device, batch_size=64)

Generation: mix-instruct_train_21000_Phi-3-mini-128k-instruct not found in cache, generating responses 🏃


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Generating responses:   0%|          | 1/329 [01:25<7:45:37, 85.18s/it]

In [15]:
from evaluation import compute_metrics
# For Phi3 (initial)
# Reported BGE: 0.80, ROUGE: 0.35
compute_metrics(responses, references, generation_name, device=device)

Evaluate: mix-instruct_train_21000_Phi-3-mini-128k-instruct_bge found in cache, loading from cache ✅
Evaluate: mix-instruct_train_21000_Phi-3-mini-128k-instruct_rouge found in cache, loading from cache ✅


{'bge': 0.808487, 'rouge': 0.21496158928534306}

In [16]:
# For LLama 3.2 (initial)
# Reported BGE: 0.73, ROUGE: 0.28
responses_llama, generation_name_llama = generate_responses(prompts, 'meta-llama/Llama-3.2-3B', ds_name, device, batch_size=64)
compute_metrics(responses_llama, references, generation_name_llama, device=device)

Generation: mix-instruct_train_21000_Llama-3.2-3B found in cache, loading from cache ✅
Evaluate: mix-instruct_train_21000_Llama-3.2-3B_bge found in cache, loading from cache ✅
Evaluate: mix-instruct_train_21000_Llama-3.2-3B_rouge found in cache, loading from cache ✅


{'bge': 0.7400304, 'rouge': 0.10360998123357584}

In [1]:
# For LLama 3.2 (finetuned)
# Reported BGE: 0.73, ROUGE: 0.28
from finetune import fine_tune_model
from data_loader import get_mix_instruct
from utility_functions.delift_se import get_delift_se_utility
from subset import create_subset, get_subset
from inference import generate_responses
from evaluation import compute_metrics



prompts, references, ds_name = get_mix_instruct("train", 210)
utility, utility_name = get_delift_se_utility(prompts, references, ds_name)
subset, subset_name = create_subset(utility, utility_name)
s_prompts, s_references = get_subset(subset, prompts, references)
prompts_val, references_val, ds_name_valid = get_mix_instruct("validation", 50)
model_dir = fine_tune_model('meta-llama/Llama-3.2-3B', prompts, references, prompts_val, references_val, subset_name)

responses_llama_ft, generation_name_llama_ft = generate_responses(prompts, model_dir, ds_name, device, batch_size=64)
compute_metrics(responses_llama_ft, references, generation_name_llama_ft, device=device)

Dataset: mix-instruct_train_210 found in cache, loading from cache ✅
Utility: mix-instruct_train_210_delift-se found in cache, loading from cache ✅
Subset: mix-instruct_train_210_delift-se_0.3 found in cache, loading from cache ✅
Dataset: mix-instruct_validation_50 found in cache, loading from cache ✅
Finetune: Fine-tuned model found in cache. Skipping Training ✅
Generation: mix-instruct_train_210_Llama-3.2-3B_mix-instruct_train_210_delift-se_0.3 not found in cache, generating responses 🏃


You are attempting to use Flash Attention 2.0 with a model not initialized on GPU. Make sure to move the model to GPU after initializing it on CPU with `model.to('cuda')`.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Generating responses: 100%|██████████| 4/4 [02:45<00:00, 41.40s/it]

Generation: mix-instruct_train_210_Llama-3.2-3B_mix-instruct_train_210_delift-se_0.3 generated and saved to cache ✅
Evaluate: mix-instruct_train_210_Llama-3.2-3B_mix-instruct_train_210_delift-se_0.3_bge found in cache, loading from cache ✅
Evaluate: mix-instruct_train_210_Llama-3.2-3B_mix-instruct_train_210_delift-se_0.3_rouge found in cache, loading from cache ✅


{'bge': 0.6989159, 'rouge': 0.1114583432053361}

In [10]:
compute_metrics([r[:int(len(r) * 0.2)] for r in responses_llama_ft], references, generation_name_llama_ft + "_temp", device='cuda:0')

Evaluate: mix-instruct_train_210_Llama-3.2-3B_mix-instruct_train_210_delift-se_0.3_temp_bge not found in cache, computing now 🏃


Computing BGE: 100%|██████████| 1/1 [00:02<00:00,  2.83s/it]


Evaluate: mix-instruct_train_210_Llama-3.2-3B_mix-instruct_train_210_delift-se_0.3_temp_bge computed and saved to cache ✅
Evaluate: mix-instruct_train_210_Llama-3.2-3B_mix-instruct_train_210_delift-se_0.3_temp_rouge not found in cache, computing now 🏃


Evaluating ROUGE: 100%|██████████| 1/1 [00:01<00:00,  1.02s/it]

Evaluate: mix-instruct_train_210_Llama-3.2-3B_mix-instruct_train_210_delift-se_0.3_temp_rouge computed and saved to cache ✅


{'bge': 0.7549735, 'rouge': 0.3130293886059544}